In [1]:
import numpy as np
import pandas as pd
import timeit

from numpy import linalg as LA

from datetime import datetime

from sklearn.model_selection import KFold

import xgboost as xgb

from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

from mlxtend.regressor import StackingRegressor
from mlxtend.data import boston_housing_data

from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pal = sns.color_palette()

/root/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#-------merge ans-------
BASE_DIR = '../../data/'

df_test = pd.read_csv(BASE_DIR+'Quora/input/test.csv')

lystdo_1_ans = pd.read_csv(BASE_DIR+'Quora/input/lystdo_ans_lstm_193_108_0.33_0.22_1.csv')
lystdo_ft_ans = pd.read_csv(BASE_DIR+'Quora/input/lystdo_ans_lstm_175_117_0.20_0.15_ft.csv')
hadry_ans = pd.read_csv(BASE_DIR+'Quora/input/hadry_ans_6.csv')

myans = pd.DataFrame()
myans['test_id'] = df_test['test_id']
myans['is_duplicate'] = lystdo_1_ans['is_duplicate']*0.1+lystdo_ft_ans['is_duplicate']*0.1+ hadry_ans['is_duplicate']*0.8

myans.to_csv(BASE_DIR+'Quora/input/my_ans_21.csv', index=False)

In [3]:
#-------load ori data--------
df_train = pd.read_csv('../../data/Quora/input/train.csv')
df_test = pd.read_csv('../../data/Quora/input/test.csv')
df_train = df_train.fillna(' ')
df_test = df_test.fillna(' ')

In [4]:
#-------load hadry feature file--------
hadry_x_train = pd.read_csv('../../data/Quora/feature/hadry_X_train.csv', header=0)
hadry_x_test  = pd.read_csv('../../data/Quora/feature/hadry_X_test.csv', header=0)

hadry_x_train=hadry_x_train.rename(columns = {'q1_q2_intersect.1':'q1_q2_intersect'})
hadry_x_train=hadry_x_train.rename(columns = {'q1_freq.1':'q1_freq'})
hadry_x_train=hadry_x_train.rename(columns = {'q2_freq.1':'q2_freq'})

hadry_x_test=hadry_x_test.rename(columns = {'q1_q2_intersect.1':'q1_q2_intersect'})
hadry_x_test=hadry_x_test.rename(columns = {'q1_freq.1':'q1_freq'})
hadry_x_test=hadry_x_test.rename(columns = {'q2_freq.1':'q2_freq'})

#--------Add ad feature------
abh_x_train = pd.read_csv('../../data/Quora/feature/Abhishek/train_features.csv', header=0)
abh_x_test  = pd.read_csv('../../data/Quora/feature/Abhishek/test_features.csv', header=0)

abh_x_train = abh_x_train.drop('euclidean_distance', axis=1)
abh_x_train = abh_x_train.drop('jaccard_distance', axis=1)
abh_x_train = abh_x_train.drop('question1', axis=1)
abh_x_train = abh_x_train.drop('question2', axis=1)

abh_x_test = abh_x_test.drop('euclidean_distance', axis=1)
abh_x_test = abh_x_test.drop('jaccard_distance', axis=1)
abh_x_test = abh_x_test.drop('question1', axis=1)
abh_x_test = abh_x_test.drop('question2', axis=1)


In [5]:
#----------merge feature------
x_train = pd.concat((hadry_x_train, abh_x_train), axis=1)
x_test  = pd.concat((hadry_x_test,  abh_x_test), axis=1)

In [6]:
print len(x_train.columns)
print len(x_train)
print len(x_test)

43
404290
2345796


In [7]:
x_train.columns

Index([u'word_match', u'jaccard', u'wc_diff', u'wc_ratio', u'wc_diff_unique',
       u'wc_ratio_unique', u'wc_diff_unq_stop', u'wc_ratio_unique_stop',
       u'same_start', u'char_diff', u'char_diff_unq_stop',
       u'total_unique_words', u'total_unq_words_stop', u'char_ratio',
       u'q1_q2_intersect', u'q1_freq', u'q2_freq', u'len_q1', u'len_q2',
       u'diff_len', u'len_char_q1', u'len_char_q2', u'len_word_q1',
       u'len_word_q2', u'common_words', u'fuzz_qratio', u'fuzz_WRatio',
       u'fuzz_partial_ratio', u'fuzz_partial_token_set_ratio',
       u'fuzz_partial_token_sort_ratio', u'fuzz_token_set_ratio',
       u'fuzz_token_sort_ratio', u'wmd', u'norm_wmd', u'cosine_distance',
       u'cityblock_distance', u'canberra_distance', u'minkowski_distance',
       u'braycurtis_distance', u'skew_q1vec', u'skew_q2vec', u'kur_q1vec',
       u'kur_q2vec'],
      dtype='object')

In [8]:
#--------Add extra feature: fasttx------
ft_x_train = pd.read_csv('../../data/Quora/feature/ft_trainX.csv', header=0)
ft_x_test = pd.read_csv('../../data/Quora/feature/ft_testX.csv', header=0)

x_train['ft_cosine']     = ft_x_train['ft_cosine']
x_train['ft_cityblock']  = ft_x_train['ft_cityblock']
x_train['ft_canberra']   = ft_x_train['ft_canberra']
x_train['ft_minkowski']  = ft_x_train['ft_minkowski']
x_train['ft_braycurtis'] = ft_x_train['ft_braycurtis']

x_test['ft_cosine']     = ft_x_test['ft_cosine']
x_test['ft_cityblock']  = ft_x_test['ft_cityblock']
x_test['ft_canberra']   = ft_x_test['ft_canberra']
x_test['ft_minkowski']  = ft_x_test['ft_minkowski']
x_test['ft_braycurtis'] = ft_x_test['ft_braycurtis']

gv_x_train = pd.read_csv('../../data/Quora/feature/gv_trainX.csv', header=0)
gv_x_test = pd.read_csv('../../data/Quora/feature/gv_testX.csv', header=0)

x_train['gv_cosine']     = gv_x_train['gv_cosine']
x_train['gv_cityblock']  = gv_x_train['gv_cityblock']
x_train['gv_canberra']   = gv_x_train['gv_canberra']
x_train['gv_minkowski']  = gv_x_train['gv_minkowski']
x_train['gv_braycurtis'] = gv_x_train['gv_braycurtis']

x_test['gv_cosine']     = gv_x_test['gv_cosine']
x_test['gv_cityblock']  = gv_x_test['gv_cityblock']
x_test['gv_canberra']   = gv_x_test['gv_canberra']
x_test['gv_minkowski']  = gv_x_test['gv_minkowski']
x_test['gv_braycurtis'] = gv_x_test['gv_braycurtis']


In [9]:
y_train = df_train['is_duplicate'].values

In [10]:
print len(x_train.columns)
print len(x_test.columns)
print len(x_train)
print len(y_train)
print len(x_test)

53
53
404290
404290
2345796


In [11]:
#------train xgb

X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=4242)

#UPDownSampling
pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]
X_train = pd.concat((neg_train, pos_train.iloc[:int(0.8*len(pos_train))], neg_train))
y_train = np.array([0] * neg_train.shape[0] + [1] * pos_train.iloc[:int(0.8*len(pos_train))].shape[0] + [0] * neg_train.shape[0])
print(np.mean(y_train))
del pos_train, neg_train

pos_valid = X_valid[y_valid == 1]
neg_valid = X_valid[y_valid == 0]
X_valid = pd.concat((neg_valid, pos_valid.iloc[:int(0.8 * len(pos_valid))], neg_valid))
y_valid = np.array([0] * neg_valid.shape[0] + [1] * pos_valid.iloc[:int(0.8 * len(pos_valid))].shape[0] + [0] * neg_valid.shape[0])
print(np.mean(y_valid))
del pos_valid, neg_valid

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 5
params['subsample'] = 0.6
params['base_score'] = 0.2
# params['scale_pos_weight'] = 0.2

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50, verbose_eval=50)
print(log_loss(y_valid, bst.predict(d_valid)))

#------TEST xgb

d_test = xgb.DMatrix(x_test)
p_test = bst.predict(d_test)
sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('../../data/Quora/input/my_ans_16.csv', index=False)

0.189752932122
0.189234677675
[0]	train-logloss:0.476034	valid-logloss:0.475378
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[50]	train-logloss:0.281815	valid-logloss:0.282516
[100]	train-logloss:0.234631	valid-logloss:0.235906
[150]	train-logloss:0.217952	valid-logloss:0.219596
[200]	train-logloss:0.210505	valid-logloss:0.212342
[250]	train-logloss:0.205289	valid-logloss:0.207451
[300]	train-logloss:0.201175	valid-logloss:0.203609
[350]	train-logloss:0.198209	valid-logloss:0.200893
[400]	train-logloss:0.19588	valid-logloss:0.198784
[450]	train-logloss:0.193898	valid-logloss:0.19712
[500]	train-logloss:0.192267	valid-logloss:0.195765
[550]	train-logloss:0.190804	valid-logloss:0.194602
[600]	train-logloss:0.189722	valid-logloss:0.193794
[650]	train-logloss:0.188643	valid-logloss:0.193023
[700]	train-logloss:0.187638	valid-logloss:0.192387
[750]	train-logloss:0.186828	valid-logloss:0